In [2]:
#using fine-tuned emotion model for sentiment analysis

In [4]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k = None, 
    device = 'cuda'
)

classifier("I love this!")

Device set to use cuda


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764602683484554},
  {'label': 'anger', 'score': 0.004419787786900997},
  {'label': 'sadness', 'score': 0.0020923942793160677},
  {'label': 'disgust', 'score': 0.0016119946958497167},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [6]:
import pandas as pd
books = pd.read_csv("books_with_categories.csv")

In [7]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [8]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548407673835754},
  {'label': 'neutral', 'score': 0.16985219717025757},
  {'label': 'sadness', 'score': 0.11640916764736176},
  {'label': 'surprise', 'score': 0.020700708031654358},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.01516129169613123},
  {'label': 'anger', 'score': 0.0039351508021354675}]]

In [12]:
predictions = classifier(books["description"][0].split(".")) #split sentiment analysis by each line

In [13]:
sorted(predictions[0], key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.009156355634331703},
 {'label': 'disgust', 'score': 0.002628473797813058},
 {'label': 'fear', 'score': 0.0681622102856636},
 {'label': 'joy', 'score': 0.047942474484443665},
 {'label': 'neutral', 'score': 0.14038565754890442},
 {'label': 'sadness', 'score': 0.002122160280123353},
 {'label': 'surprise', 'score': 0.7296026349067688}]

In [20]:
import numpy as np

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_prediction = sorted(prediction, key=lambda x:x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_prediction[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [21]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [02:11<00:00, 39.56it/s]


In [23]:
emotion_scores

{'anger': [0.0641336739063263,
  0.6126192808151245,
  0.0641336739063263,
  0.35148337483406067,
  0.0814124196767807,
  0.23222456872463226,
  0.5381830930709839,
  0.0641336739063263,
  0.30067017674446106,
  0.0641336739063263,
  0.0641336739063263,
  0.6115738153457642,
  0.10621541738510132,
  0.0641336739063263,
  0.07239239662885666,
  0.0958634540438652,
  0.162159264087677,
  0.0641336739063263,
  0.08193837851285934,
  0.22345967590808868,
  0.10715226083993912,
  0.0641336739063263,
  0.0641336739063263,
  0.0641336739063263,
  0.39636626839637756,
  0.06472659111022949,
  0.0641336739063263,
  0.0641336739063263,
  0.0641336739063263,
  0.3029331564903259,
  0.06683346629142761,
  0.0641336739063263,
  0.0641336739063263,
  0.16889651119709015,
  0.0641336739063263,
  0.0641336739063263,
  0.0641336739063263,
  0.07910335808992386,
  0.11856506764888763,
  0.0641336739063263,
  0.5636680126190186,
  0.9171825051307678,
  0.06835665553808212,
  0.22059908509254456,
  0.0641

In [24]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn
emotions_df.head(5)

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928168,0.932797,0.646217,0.967158,0.729603,9780002005883
1,0.612619,0.348285,0.942528,0.704421,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767236,0.549477,0.111690,0.078766,9780006178736
3,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078766,9780006280934


In [26]:
books = pd.merge(books, emotions_df, on = "isbn13")
books.head(2)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928168,0.932797,0.646217,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348285,0.942528,0.704421,0.887939,0.111690,0.252545


In [27]:
books.to_csv("books_with_emotions.csv", index = False)